In [51]:
import tensorflow as tf

In [52]:
tf.reset_default_graph()

In [54]:
# defining a logistic regression function that can be used other places too
def logistic_regression(X, y, initializer=None, seed=42, learning_rate=0.01):
    # identifying the shape of the input
    num_features = int(X.get_shape()[1])
    num_classes = int(y.get_shape()[1])
    #n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("logistic_regression"):
        with tf.name_scope("model"):
            # initializing the weights, default
            if initializer is None:
                initializer = tf.random_normal([num_features,num_classes ], stddev=0.01, seed=seed)
            
            # defining the weights
            weights = tf.Variable(initializer, name="weights")
            # initializing zero vectors
            bias = tf.Variable(tf.zeros([1, num_classes]), name="bias")
            # fundamental function to be added to softmax
            logits = tf.matmul(X, weights, name="logits") + bias 
            # softmax function
            y_predicted = tf.nn.softmax(logits, name="softmax")
            
        with tf.name_scope("train"):
            # softmax regression with cross entropy loss
            entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y, name='cross_entropy_loss')
            # final loss
            loss = tf.reduce_mean(entropy)
            # using adam optimizer to minimize loss
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
            
            # loss summary for tensorboard
            loss_summary = tf.summary.scalar('cross_entropy', loss)

        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return y_predicted, loss, optimizer, loss_summary, init, saver

In [55]:
# Creating a function for log directories of tensorboard
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "./{}/{}/".format(root_logdir, name)

In [56]:
logdir = log_dir("logregression")


# Creating placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# each lable is one hot vector.
X = tf.placeholder(tf.float32, shape=([None, 784]), name='X_placeholder') 
Y = tf.placeholder(tf.int32, shape=([None, 10]), name='Y_placeholder')

# Define paramaters for the model
learning_rate = 0.001
batch_size = 128
n_epochs = 20



In [57]:
# Creating the computation graph

y_pred, loss, optimizer, loss_summary, init, saver= logistic_regression(X, Y)


# Writing it to tensorboard
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
